In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
from torchvision import models, transforms
from PIL import Image
import pickle

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# Get the current directory
current_dir = Path.cwd()

encoded_dir = current_dir.parent / "data" / "encoded"
images_dir = current_dir.parent / "items_images"
pre_path = current_dir.parent / "data" / "pre_process"

In [ ]:
folder = os.listdir(images_dir)
len(folder)

In [ ]:
# Load the item_mapping
with open(pre_path / 'item_mapping.pkl', 'rb') as f:
    item_mapping = pickle.load(f)

In [ ]:
len(item_mapping)

# Create the embeddings dict using ResNet

In [ ]:
# Load ResNet50 model
model = models.resnet50(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Remove final classification layer
model.to(device)
model.eval()

In [ ]:
# Define preprocessing transformations
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize to ResNet's input size
    transforms.ToTensor(),          # Convert to tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])  # Normalize as per ResNet

In [ ]:
# Initialize the dictionary to store embeddings
image_embeddings = {}

i = 0
# Process images
for image_file in images_dir.iterdir():
    if image_file.suffix.lower() in ['.jpg', '.png', '.jpeg']:  # Check for valid image extensions
        try:
            parent_asin = image_file.stem  # Extract parent_asin from the filename
            item_idx = item_mapping.get(parent_asin)  # Map parent_asin to item_idx
            image = Image.open(image_file).convert("RGB")  # Ensure image is RGB
            image_tensor = image_transforms(image).unsqueeze(0).to(device)  # Add batch dimension and move to GPU
            with torch.no_grad():
                embedding = model(image_tensor).squeeze().flatten()  # Extract embeddings and Flatten the output to a vector

            image_embeddings[item_idx] = embedding  # Store the embedding with item_idx as the key

        except Exception as e:
            print(f"Error processing {image_file.name}: {e}")

        i += 1
        if i % 100 == 0:
            print(f"Processed {i} images")

In [ ]:
image_embeddings[0].shape

In [ ]:
# Ensure all items are present in the image_embeddings dictionary
embedding_size = 2048  # Size of the embedding
for item_idx in range(len(item_mapping)):
    if item_idx not in image_embeddings:
        # Add a zero tensor for missing items
        image_embeddings[item_idx] = torch.zeros(embedding_size).to(device)

print(f"Updated image_embeddings to contain all {len(item_mapping)} items.")

In [ ]:
# Check if all tensors in image_embeddings are on the same device
def check_tensors_device(tensor_dict):
    devices = {tensor.device for tensor in tensor_dict.values()}
    if len(devices) == 1:
        print(f"All tensors are on the same device: {devices.pop()}")
    else:
        print(f"Tensors are on multiple devices: {devices}")

# Example usage
check_tensors_device(images_embeddings)

In [ ]:
# Move all tensors in image_embeddings to the "cuda" device
def move_tensors_to_cuda(tensor_dict):
    target_device = torch.device("cuda")
    for key, tensor in tensor_dict.items():
        if tensor.device != target_device:
            tensor_dict[key] = tensor.to(target_device)
    print(f"All tensors have been moved to {target_device}")

# Example usage
move_tensors_to_cuda(images_embeddings)

In [ ]:
# Save embeddings dictionary
output_file = encoded_dir / "images_encodings.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(image_embeddings, f)

print(f"Image embeddings saved to {output_file}")

# Read the embeddings dict

In [ ]:
encoded_images_file = encoded_dir / "images_encodings.pkl"

with open(encoded_images_file, 'rb') as f:
    images_embeddings = pickle.load(f)

In [ ]:
images_embeddings[0].shape

In [ ]:
len(images_embeddings)

In [ ]:
images_embeddings

In [ ]:
# Save embeddings dictionary
output_file = encoded_dir / "images_encodings.pkl"
with open(output_file, 'wb') as f:
    pickle.dump(images_embeddings, f)

print(f"Image embeddings saved to {output_file}")

In [ ]:
encoded_images_file = encoded_dir / "images_encodings.pkl"

with open(encoded_images_file, 'rb') as f:
    images_embeddings = pickle.load(f)